<a href="https://www.kaggle.com/code/hetarthchopra/sentencetransformers-lecr-first-model?scriptVersionId=115206532" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!cp -r ../input/sentence-transformers-dataset/sentence-transformers /tmp/st
!pip install /tmp/st

Processing /tmp/st
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126135 sha256=24311211049bc9b816cfb8aa26b0fa0685dc08da1ad6f27ac1b641d483452896
  Stored in directory: /tmp/pip-ephem-wheel-cache-n9prc68z/wheels/07/90/8c/ea25d1a415b340d7b89c6ceb0aec3184514860bcf8813763ae
Successfully built sentence-transformers


Code Inspired from https://www.kaggle.com/code/ssarkar445/le-curriculum-sbert-baseline/notebook and https://www.kaggle.com/code/hetarthchopra/sentence-transformer-nli

In [2]:
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

## Build dataset

In [3]:
content = pd.read_csv('/kaggle/input/learning-equality-curriculum-recommendations/content.csv')
correlations = pd.read_csv('/kaggle/input/learning-equality-curriculum-recommendations/correlations.csv')
topics = pd. read_csv('/kaggle/input/learning-equality-curriculum-recommendations/topics.csv')
sample_submission = pd.read_csv('/kaggle/input/learning-equality-curriculum-recommendations/sample_submission.csv')

In [4]:
topics = topics.rename(columns={'id':'topic_id'})
content = content.rename(columns={'id':'topic_id'})
filtered_topics = pd.merge(sample_submission, topics, on ='topic_id')
filtered_topics.drop(columns = ['content_ids'])

,topic_id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
1,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
2,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True
3,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
4,t_4054df11a74e,Flow Charts: Logical Thinking?,This lesson is focused on flow charts. It supp...,6e3ba4,source,2,en,t_acbbd893e6af,True


In [5]:
# impute missing data 
filtered_topics['title'] = filtered_topics['title'].fillna('')
filtered_topics['description'] = filtered_topics['description'].fillna('')
content['title'] = content['title'].fillna('')
content['description'] = content['description'].fillna('')

In [6]:
filtered_topics['combined_text'] = filtered_topics['title']+', '+filtered_topics['description']
content['combined_text'] = content['title']+', '+content['description']

In [7]:
cleaned_topic = filtered_topics[['topic_id','combined_text']]
cleaned_content = content[['topic_id','combined_text']]

## Get Sentence Embeddings

In [8]:
model = SentenceTransformer('/kaggle/input/allmpnetbasev2/all-mpnet-base-v2')

In [ ]:
# generate all embeddings of the content data, as a search space for semantic search
embeddings = model.encode(cleaned_content['combined_text'], convert_to_tensor=True)

Batches:   0%|          | 0/4814 [00:00<?, ?it/s]

In [ ]:
predictions = []
top_k = 4
for i in cleaned_topic['combined_text']:
    topic_embedding = model.encode(i, convert_to_tensor=True)
    cosine_sim = util.cos_sim(topic_embedding, embeddings)[0]
    top_results = torch.topk(cosine_sim, k=top_k)
    predictions.append(top_results[1].cpu().numpy())

In [ ]:
predictions

## Submit

In [ ]:
combined_preds = []
for x in predictions:
    ids = cleaned_content['topic_id'].iloc[x]
    combined_preds.append(' '.join(ids))

In [ ]:
sample_submission['content_ids'] = combined_preds
sample_submission.to_csv('submission.csv',index=False)
sample_submission.head()